In [1]:
import os
import itertools

import pandas as pd
import xarray as xr

import climatools.lblnew.pipeline as pipeline
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.bestfit_params as bestfit

import importlib
importlib.reload(setup_bestfit)

<module 'climatools.lblnew.setup_bestfit' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/setup_bestfit.py'>

In [6]:
def define_param():
    '''
    Define a particular lblnew best-fit run.
    '''    
    molecule = 'co2'
    band = '3a'

    dv = 0.001
    nv = 1000

    ref_pts = [(1, 250), (10, 250), (500, 250)]
    ng_refs = [3, 2, 4]
    ng_adju = [0, 0]
    klin = 6.375563e-24
    option_wgt_k = 1
    wgt = [(.7, .3, .7), 
           (.7, .6), 
           (.4, .5, .8, .95)]

    option_wgt_flux = 2
    w_diffuse = [(1.6, 1.6, 1.7), 
                 (1.75, 1.75), 
                 (1.55, 1.65, 1.6, 1.85)]
    
    option_compute_ktable = 0
    option_compute_btable = 0
    
    atmpro = 'mls'
    tsfc = 294
    
    commitnumber = '1996322'    

    param = dict(molecule=molecule,
                 band=band, 
                 dv=dv, nv=nv,
                 ref_pts=ref_pts, ng_refs=ng_refs, ng_adju=ng_adju,
                 klin=klin, option_wgt_k=option_wgt_k, wgt=wgt,
                 option_wgt_flux=option_wgt_flux, w_diffuse=w_diffuse,
                 option_compute_ktable=option_compute_ktable,
                 option_compute_btable=option_compute_btable,
                 atmpro=atmpro, tsfc=tsfc,
                 commitnumber=commitnumber)
  
    return param



def fpath(param):
    path = os.path.join(pipeline.get_dir_case(param, 
                                               setup=setup_bestfit),
                         'dgdgs.dat')
    return path



def load_dgdgs(path):
    path = '/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/co2/band03a_wn_540_620/nv_1000/dv_0.001/ng_9/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_3__2__4/ng_adju_0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_6.375563e-24/atmpro_mls/wgt_k_1/wgt_0.7_0.3_0.7__0.7_0.6__0.4_0.5_0.8_0.95/wgt_flux_2/w_diffuse_1.6_1.6_1.7__1.75_1.75__1.55_1.65_1.6_1.85/option_compute_ktable_0/crd_1996322/dgdgs.dat'
    df = pd.read_csv(path, sep=r'\s+')

    df = df.set_index('g')    
    return df



def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4):
    '''
    '''
    strfmt = '{:15.6e}'
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines



def dgdgs_to_F77(dgdgs):
    lines = vector_to_F77list(dgdgs, num_values_per_line=3)
    
    rlines = []
    rlines.append(5 * ' ' + '&/' + lines[0])
    for line in lines[1:-1]:
        rlines.append(5 * ' ' + '&' + line)
    rlines.append(5 * ' ' + '&' + lines[-1] + '/')
    
    fortran = '\n'.join(rlines)
    return fortran


def script():
    dgdgs = load_dgdgs(fpath(define_param()))
    return dgdgs_to_F77(dgdgs['dgs'])

In [7]:

print(script())


     &/   7.177335e-04,   1.596256e-03,   7.011057e-03,
     &   1.205621e-02,   4.641767e-02,   6.328772e-02,
     &   2.638414e-01,   2.288560e-01,   3.762160e-01/
